In [4]:
import numpy as np
import pandas as pd
import cv2
import csv
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
%matplotlib inline

In [25]:
import os
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.utils.data import Dataset
from torchvision import transforms

In [10]:
def check_point(ax, ay, bx, by, cx, cy):
    bax = bx - ax
    bay = by - ay
    bcx = cx - bx
    bcy = cy - by
    if bax * bcy - bay * bcx <= 0:
        return True
    return False

In [11]:
def check_in_polygon(x1, y1, x2, y2, x3, y3, x4, y4, px, py):
    if check_point(x1, y1, px, py, x2, y2) and check_point(x2, y2, px, py, x3, y3) and \
    check_point(x3, y3, px, py, x4, y4) and check_point(x4, y4, px, py, x1, y1):
        return True
    else:
        return False

In [20]:
def make_new_df(dest, image_to_shape, mask_to_shape):
    train_df = pd.read_csv(
        '/DATA/ichuviliaeva/ocr_data/barcodes/CourseOCRTask3/Train/markup_wo_inverted.csv', 
        encoding = 'utf-16', header=None
    )
    
    orig_files = []
    resized_files, mask_files = [], []
    ws, hs = [], []
    x1_s, y1_s, x2_s, y2_s, x3_s, y3_s, x4_s, y4_s = [], [], [], [], [], [], [], []
    
    for i in tqdm(range(len(train_df))):
        file = train_df.iloc[i][0]
        orig_files.append(file)
        
        x1, y1, x2, y2, x3, y3, x4, y4 = train_df.iloc[i][2:10]
        image = Image.open('/DATA/ichuviliaeva/ocr_data/barcodes/CourseOCRTask3/Train/Images/' + file)
        
        w, h = image.size
        ws.append(w)
        hs.append(h)
        
        image = transforms.Resize((image_to_shape))(image)
        image.save(dest + '/Images/' + file.split('.')[0] + '_resized.png')
        resized_files.append(file.split('.')[0] + '_resized.png')
        
        mask = torch.zeros((1, mask_to_shape[0], mask_to_shape[1]))
        
        x1 = mask_to_shape[0] * x1 / w
        y1 = mask_to_shape[1] * y1 / h
        x2 = mask_to_shape[0] * x2 / w
        y2 = mask_to_shape[1] * y2 / h
        x3 = mask_to_shape[0] * x3 / w
        y3 = mask_to_shape[1] * y3 / h
        x4 = mask_to_shape[0] * x4 / w
        y4 = mask_to_shape[1] * y4 / h
        
        x1_s.append(x1)
        y1_s.append(y1)
        x2_s.append(x2)
        y2_s.append(y2)
        x3_s.append(x3)
        y3_s.append(y3)
        x4_s.append(x4)
        y4_s.append(y4)
        
        for i in range(mask_to_shape[0]):
            for j in range(mask_to_shape[1]):
                if check_in_polygon(y1, x1, y4, x4, y3, x3, y2, x2, i, j):
                    mask[0][i][j] = 1
                else:
                    mask[0][i][j] = 0
        torch.save(mask, dest + '/masks/' + file.split('.')[0] + '_mask.pt')
        mask_files.append(file.split('.')[0] + '_mask.pt')
        
    res_df = pd.DataFrame()
    res_df['original_file'] = orig_files
    res_df['resized_file'] = resized_files
    res_df['mask_file'] = mask_files
    res_df['original_w'] = ws
    res_df['original_h'] = hs
    res_df['x1'] = x1_s
    res_df['y1'] = y1_s
    res_df['x2'] = x2_s
    res_df['y2'] = y2_s
    res_df['x3'] = x3_s
    res_df['y3'] = y3_s
    res_df['x4'] = x4_s
    res_df['y4'] = y4_s
    res_df.to_csv(dest + 'resized_table.csv')
    return res_df

In [21]:
res_df = make_new_df('/DATA/ichuviliaeva/ocr_data/barcodes/resized', (512, 512), (76, 76))

  0%|          | 0/8150 [00:00<?, ?it/s]

In [23]:
display(res_df)

,original_file,resized_file,mask_file,original_w,original_h,x1,y1,x2,y2,x3,y3,x4,y4
0,fc272e4092ccb147a31e66bc1d10fbfb.png,fc272e4092ccb147a31e66bc1d10fbfb_resized.png,fc272e4092ccb147a31e66bc1d10fbfb_mask.pt,2129,901,19.205261,20.918979,56.759042,18.978912,57.008924,55.924528,18.991076,57.021088
1,8cde8cb1b54bca37c9347baf74157d22.png,8cde8cb1b54bca37c9347baf74157d22_resized.png,8cde8cb1b54bca37c9347baf74157d22_mask.pt,496,1724,57.000000,19.088167,55.161290,56.823666,19.000000,57.000000,22.524194,19.000000
2,70a9be0887eae17eabd70c8e607f963f.png,70a9be0887eae17eabd70c8e607f963f_resized.png,70a9be0887eae17eabd70c8e607f963f_mask.pt,1632,917,19.000000,19.062159,57.000000,18.979280,57.000000,57.020720,19.279412,56.606325
3,a029e49dff95a15d2cd396d8f62220c8.png,a029e49dff95a15d2cd396d8f62220c8_resized.png,a029e49dff95a15d2cd396d8f62220c8_mask.pt,1340,596,19.000000,19.000000,57.000000,19.765101,57.000000,55.852349,19.000000,57.000000
4,82ff01909b6d215170dfc26c7be43074.png,82ff01909b6d215170dfc26c7be43074_resized.png,82ff01909b6d215170dfc26c7be43074_mask.pt,1856,949,19.204741,18.979979,56.836207,20.821918,57.000000,57.020021,19.000000,55.738672
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8145,c532771808e76e333ba9e959853886f3.png,c532771808e76e333ba9e959853886f3_resized.png,c532771808e76e333ba9e959853886f3_mask.pt,860,276,7.600000,14.869565,54.967442,14.594203,54.967442,55.623188,7.688372,55.623188
8146,186338ad8f3ff17b6082c023f24563f0.png,186338ad8f3ff17b6082c023f24563f0_resized.png,186338ad8f3ff17b6082c023f24563f0_mask.pt,877,287,9.445838,11.651568,57.541619,14.564460,57.888255,54.550523,9.272520,50.843206
8147,5bac8f6c9bd5e99f92eb396442974c4e.png,5bac8f6c9bd5e99f92eb396442974c4e_resized.png,5bac8f6c9bd5e99f92eb396442974c4e_mask.pt,905,274,10.161326,7.489051,59.120442,3.605839,59.120442,46.598540,10.077348,51.868613
8148,6247db3e424148a859ebf2180f1dcb6f.png,6247db3e424148a859ebf2180f1dcb6f_resized.png,6247db3e424148a859ebf2180f1dcb6f_mask.pt,220,587,54.581818,10.487223,57.000000,67.066440,21.418182,67.454855,19.000000,10.616695


In [37]:
class BarcodeDataset(Dataset):

    def __init__(self, data_path):
        self.data_path = data_path
        self.df = pd.read_csv(data_path + '/resizedresized_table.csv')
        self.image_to_shape = (512, 512)
        self.mask_to_shape = (76, 76)
        
    def __getitem__(self, index):
        file = self.df.iloc[index]['resized_file']
        image = Image.open(self.data_path + '/Images/' + file)
        image = transforms.ToTensor()(image)
        
        mask = torch.load(self.data_path + '/masks/' + self.df.iloc[index]['mask_file'])
        return image, mask
    
    def __len__(self) -> int:
        return len(self.df)

In [44]:
train_dataset = BarcodeDataset('/DATA/ichuviliaeva/ocr_data/barcodes/resized')
train_loader = data.DataLoader(train_dataset, batch_size = 64, shuffle = True)

In [45]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [46]:
class ZagaynovNet(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(ZagaynovNet, self).__init__()
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.net = nn.Sequential(
            nn.Conv2d(in_channels = self.in_channels, out_channels = self.hidden_channels, kernel_size = 3, 
                      padding = 1, stride = 2),
            nn.ReLU(),
            nn.Conv2d(in_channels = self.hidden_channels, out_channels = self.hidden_channels, kernel_size = 3, 
                      padding = 1, stride = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = self.hidden_channels, out_channels = self.hidden_channels, kernel_size = 3, 
                      padding = 1, stride = 2),
            nn.ReLU(),
            nn.Conv2d(in_channels = self.hidden_channels, out_channels = self.hidden_channels, kernel_size = 3, 
                      padding = 1, stride = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = self.hidden_channels, out_channels = self.hidden_channels, kernel_size = 3, 
                      padding = 1, stride = 1, dilation = 2),
            nn.ReLU(),
            nn.Conv2d(in_channels = self.hidden_channels, out_channels = self.hidden_channels, kernel_size = 3, 
                      padding = 1, stride = 1, dilation = 4),
            nn.ReLU(),
            nn.Conv2d(in_channels = self.hidden_channels, out_channels = self.hidden_channels, kernel_size = 3, 
                      padding = 1, stride = 1, dilation = 8),
            nn.ReLU(),
            nn.Conv2d(in_channels = self.hidden_channels, out_channels = self.hidden_channels, kernel_size = 3, 
                      padding = 1, stride = 1, dilation = 16),
            nn.ReLU(),
            nn.Conv2d(in_channels = self.hidden_channels, out_channels = self.hidden_channels, kernel_size = 3, 
                      padding = 1, stride = 1, dilation = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels = self.hidden_channels, out_channels = self.out_channels, kernel_size = 3, 
                      padding = 1, stride = 1, dilation = 1),
            nn.Sigmoid(),
        )
        
    def forward(self, x):
        return self.net(x)

In [47]:
model = ZagaynovNet(3, 24, 1).to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.BCELoss()

In [48]:
def train(model, epoch, optimizer, criterion, train_dataloader):
    for e in range(epoch):
        sum_loss = 0
        for image, mask in tqdm(train_dataloader):
            
            image = image.to(device)
            mask = mask.to(device)
            logits = model(image)
            # print('logits.shape = ', logits.shape)
            loss = criterion(logits, mask)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            sum_loss += loss.item()
            
        print('epoch {} ended mean loss:{:.3f}'.format(e, sum_loss / len(train_dataloader)))
            

In [49]:
train(model, 10, optimizer, criterion, train_loader)

  0%|          | 0/128 [00:00<?, ?it/s]

epoch 0 ended mean loss:0.442


  0%|          | 0/128 [00:00<?, ?it/s]

epoch 1 ended mean loss:0.207


  0%|          | 0/128 [00:00<?, ?it/s]

...

epoch 7 ended mean loss:0.126


  0%|          | 0/128 [00:00<?, ?it/s]

epoch 8 ended mean loss:0.120


  0%|          | 0/128 [00:00<?, ?it/s]

epoch 9 ended mean loss:0.118


In [51]:
torch.save(model.state_dict, 'detector_512_epoch_9.pth') 

In [94]:
def train(model, epoch, optimizer, criterion, train_dataloader):
    for e in range(epoch):
        sum_loss = 0
        for image, mask in tqdm(train_dataloader):
            
            image = image.to(device)
            mask = mask.to(device)
            logits = model(image)
            # print('logits.shape = ', logits.shape)
            loss = criterion(logits, mask)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            sum_loss += loss.item()
            
        print('epoch {} ended mean loss:{:.3f}'.format(e, sum_loss / len(train_dataloader)))
        if e % 10 == 0:
            torch.save(model.state_dict, 'detector_512_epoch_' + str(e) + '.pth') 
    return model

In [96]:
model = train(model, 70, optimizer, criterion, train_loader)

  0%|          | 0/128 [00:00<?, ?it/s]

epoch 0 ended mean loss:0.116


  0%|          | 0/128 [00:00<?, ?it/s]

epoch 1 ended mean loss:0.112


  0%|          | 0/128 [00:00<?, ?it/s]

...

epoch 67 ended mean loss:0.085


  0%|          | 0/128 [00:00<?, ?it/s]

epoch 68 ended mean loss:0.084


  0%|          | 0/128 [00:00<?, ?it/s]

epoch 69 ended mean loss:0.084


In [105]:
torch.save(model.state_dict, 'detector_512_epoch_69.pth') 

In [100]:
optimizer2 = optim.Adam(model.parameters(), lr = 0.0001)

In [102]:
def train2(model, epoch, optimizer, criterion, train_dataloader):
    start_e = 70
    for e in range(epoch):
        sum_loss = 0
        for image, mask in tqdm(train_dataloader):
            
            image = image.to(device)
            mask = mask.to(device)
            logits = model(image)
            # print('logits.shape = ', logits.shape)
            loss = criterion(logits, mask)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            sum_loss += loss.item()
            
        print('epoch {} ended mean loss:{:.3f}'.format(start_e + e, sum_loss / len(train_dataloader)))
        if epoch % 10 == 0 or e == epoch - 1:
            torch.save(model.state_dict, 'detector_512_epoch_' + str(start_e + e) + '.pth') 
    return model

In [ ]:
model = train2(model, 70, optimizer2, criterion, train_loader)

In [179]:
class BarcodeTestDataset(Dataset):

    def __init__(self, data_path):
        self.data_path = data_path
        self.files_list = os.listdir(self.data_path + '/Images')
        self.image_to_shape = (512, 512)
        
    def __getitem__(self, index):
        file = self.files_list[index]
        image = Image.open(self.data_path + '/Images/' + file)
        w, h = image.size
        image = transforms.Resize((self.image_to_shape))(image)
        image = transforms.ToTensor()(image)
        return image, w, h, file
    
    def __len__(self) -> int:
        return len(self.files_list)

In [180]:
test_dataset = BarcodeTestDataset('/DATA/ichuviliaeva/ocr_data/barcodes/CourseOCRTask3/Test')
test_loader = data.DataLoader(test_dataset, batch_size = 16, shuffle = False)

In [233]:
ans_x1, ans_y1, ans_x2, ans_y2, ans_x3, ans_y3, ans_x4, ans_y4 = [], [], [], [], [], [], [], []
ans_file = []
def predict(model, test_loader):
    for image, w, h, file in tqdm(test_loader):
        logits = model(image.to(device))
        mask_bool = 1 * (logits > 0.5)
        mask_cpu = mask_bool.detach().cpu().numpy().astype(np.uint8)
        
        for t in range(mask_cpu.shape[0]):
            ans_file.append(file[t])
            analysis = cv2.connectedComponentsWithStats(mask_cpu[t][0], 4, cv2.CV_32S)
            (totalLabels, label_ids, values, centroid) = analysis
            
            max_area = 0
            best_comp = -1
            for i in range(totalLabels):
                area = values[i, cv2.CC_STAT_AREA]  
                if area > max_area:
                    max_area = area
                    best_comp = i
            
            bestMask = (label_ids == best_comp).astype("uint8") * 255
            contours, hierarchy = cv2.findContours(bestMask, 1, 2)
            rect = cv2.minAreaRect(contours[0])
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            
            ans_x1.append(int(box[0][0] * w[t] / 76))
            ans_y1.append(int(box[0][1] * h[t] / 76))
            ans_x2.append(int(box[1][0] * w[t] / 76))
            ans_y2.append(int(box[1][1] * h[t] / 76))
            ans_x3.append(int(box[2][0] * w[t] / 76))
            ans_y3.append(int(box[2][1] * h[t] / 76))
            ans_x4.append(int(box[3][0] * w[t] / 76))
            ans_y4.append(int(box[3][1] * h[t] / 76))

In [234]:
predict(model, test_loader)

  0%|          | 0/7 [00:00<?, ?it/s]

In [237]:
res_df = pd.DataFrame()
res_df[0] = ans_file
res_df[1] = [232323232] * len(ans_file)
res_df[2] = ans_x1
res_df[3] = ans_y1
res_df[4] = ans_x2
res_df[5] = ans_y2
res_df[6] = ans_x3
res_df[7] = ans_y3
res_df[8] = ans_x4
res_df[9] = ans_y4
res_df[10] = [101010101010101010101] * len(ans_file)

In [238]:
display(res_df)

,0,1,2,3,4,5,6,7,8,9,10
0,1105913212699e2e8a558191113acbd7.png,232323232,290,529,836,375,989,1323,426,1478,101010101010101010101
1,c683d3627b6766ef1acd6efd52316be8.png,232323232,615,3722,615,1258,2791,1258,2791,3722,101010101010101010101
2,8119dd3a93dbf4e27cc3ba7262bd311e.png,232323232,1463,664,1569,664,1569,687,1463,687,101010101010101010101
3,40ed826ebe0916f70999c831fc078faf.png,232323232,850,475,850,475,850,457,850,457,101010101010101010101
4,f534d421c7e36d6133a5e9031a7e2dd5.png,232323232,232,1997,232,386,710,386,710,1997,101010101010101010101
...,...,...,...,...,...,...,...,...,...,...,...
95,f9bf5d96a0cf5d64aac4eb5806554224.png,232323232,639,1266,639,466,1826,466,1826,1266,101010101010101010101
96,97b2e27686dbb54b20cfbf9696d430f1.png,232323232,418,453,1129,427,1170,1434,459,1485,101010101010101010101
97,e083bf4d3c80b00698abdf7b4e769a8c.png,232323232,284,231,837,191,884,503,331,533,101010101010101010101
98,48e4ee3749f5e786d2160d5081a4640f.png,232323232,168,252,168,79,504,79,504,252,101010101010101010101


In [256]:
with open('answer.csv', 'w', encoding='utf-16') as f:
    writer = csv.writer(f)
    for i in range(len(res_df)):
        writer.writerow(res_df.iloc[i])

In [227]:
!pip install -e .

/bin/bash: /home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/cv2/../../../../lib/libtinfo.so.6: no version information available (required by /bin/bash)
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/ichuviliaeva/ocr_course/course_ocr/task3
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 6.2 MB/s eta 0:00:0000:0100:01
  Building editable for course_ocr_t3 (pyproject.toml) ... done
  Created wheel for course_ocr_t3: filename=course_ocr_t3-0.1.1-py3-none-any.whl size=1204 sha256=6c85cc03abd93e7db7d009b622d63af54f94a589e3ff38c32c7c2baacaebca23
  Stored in directory: /tmp/pip-ephem-wheel-cache-d5o1rwe1/wheels/ff/0f/62/c58c809f14b9f622c65e7c7a61f5b94a2819377736eeff7828
Successfully built course_ocr_t3

In [258]:
!python3 -m course_ocr_t3.evaluate

/bin/bash: /home/ichuviliaeva/miniconda3/envs/ocr_course/lib/python3.8/site-packages/cv2/../../../../lib/libtinfo.so.6: no version information available (required by /bin/bash)
Checking answer (/home/ichuviliaeva/ocr_course/course_ocr/task3/answer.csv) against markup(/home/ichuviliaeva/ocr_course/course_ocr/task3/markup.csv)
recognition_accuracy=0.0
detection_result=0.77
score=0.26949999999999996
